In [1]:
from example.crypto_store import InMemCryptoStore
from eventsourcing.encryption import CryptoRepository
from eventsourcing.repositories import EventStoreRepository
from example.event_store import InMemEventStore
from example.user import User
from datetime import date
from example.guid import guid

### Initialize the in-memory encryption key store, eventstore, and repositories

In [2]:
crypto_store = InMemCryptoStore()
event_store = InMemEventStore()

CryptoRepository.crypto_store = crypto_store
user_repo = EventStoreRepository[User](event_store, User)

### Create a user and show that its attributes are clear

In [3]:
user = User(guid(), "Paul", "Boulanger", date(1997,2,18))

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())


56b6cf99-5be5-44f5-bac8-2316631b051e Paul Boulanger 1997-02-18


### Save the user in the eventstore

The eventstore and key store are both empty

In [4]:
print(event_store.current)
print(crypto_store.store)

{}
{}


Saving the user in the eventstore

In [5]:
await user_repo.save(user,0)


The events related to the user are saved in the eventstore and an encryption key linked to the user was created and stored in the encryption key store

In [6]:
print(event_store.current)
print(crypto_store.store)

{'user-56b6cf99-5be5-44f5-bac8-2316631b051e': [<example.event_store.EventDescriptor object at 0x00000158B8DE0D10>]}
{'56b6cf99-5be5-44f5-bac8-2316631b051e': b'0U9TxOPPjQ7VbCdUanRvVOK29l8aNUty_Aa3k-7de7Y='}


### Showing the encryption

The personnal data (in this case: `first_name`, `last_name`, `month_of_birth` and `day_of_birth`) are encrypted in the eventstore

In [7]:
event_store.current[User.to_stream_id(user.id)][0].event_data

'{"id": "56b6cf99-5be5-44f5-bac8-2316631b051e", "first_name": "encrypted_gAAAAABmjiWjE4iU5WRBjpy4i-68coVHqrlC11vEOOu-23G5U2Q9MmDBxNzog8cDIMwrDKEY_CQ3_cBRN9CvYsjkr3LaXjNHsw==", "last_name": "encrypted_gAAAAABmjiWjwlsnRA-PLxphp9jLf0l0NpEmk7tHoOoKJBJLQQc_GByvCnQKO6JvWV0uSGKJ4fLNt8rtOsU7pBJnZ_qXiUZsSw==", "year_of_birth": 1997, "month_of_birth": "encrypted_gAAAAABmjiWjslk0xFbUGwBWje6gdzlL35eA1xyMkMmvv-83slmNE-fiQtfHnIV86cU-saAo1FXVo6KW8stQ4CBzHcWLSJHQ-A==", "day_of_birth": "encrypted_gAAAAABmjiWjtsyGutDNc5m1qccCq9u-lg05XW149l9LfFByetNZ1PwrxpRRPK8be5sJEst3ejMeKu59OjOOM-moT22SusQYtw=="}'

Since there is an encryption key, we can retrieve the personnal informations of the user in a decrypted state

In [8]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

InvalidToken: 

We delete the user's encryption key

In [ ]:
crypto_store.remove(user.id)

Now the personnal informations are still encrypted

In [ ]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='a8854ba1-527f-47f6-807c-8dc07ea4c42a', first_name='gAAAAABmjiRvJT59nOszJjT84fJHfUW3bOWDsk1a0na7VyLeDh6dX_TzhyvJjuXBm_f8DyyXz5NhnV-g6zInzksfSg33yXmqlw==', last_name='gAAAAABmjiRvdNjOihGYQRjg4dh8QpFas0Qs-xuXwQSP8lwOxetAIenSTUq2m_XZ3oB6l1TjyjCtZWDZ_RR-NN1F18bGeKgvlw==', year_of_birth=1997, month_of_birth='gAAAAABmjiRvM8eeEeInchs1Th5X7VjwPsLmeaXhp_QUYXz9ZzhEC8GfKQlOVyZQwq3hv7QqUGgFn9H7hnJFpCkEVEdHbUkRhA==', day_of_birth='gAAAAABmjiRv2Pkt3Wc6vp6xqmMObu6Qbh0pziz55-bA6Hj4VU5Dak_DbFeoawrSvcQR3lVT7eZ-FH4R2WT2TULw4JKNkpyr1w==')]

When we load our user from the eventstore, the personnal information are encrypted or replaced by fake information

In [ ]:
new_user = User()
new_user.loads_from_history(events)
print(new_user.id, new_user.first_name, new_user.last_name, new_user.date_of_birth.isoformat())

a8854ba1-527f-47f6-807c-8dc07ea4c42a gAAAAABmjiRvJT59nOszJjT84fJHfUW3bOWDsk1a0na7VyLeDh6dX_TzhyvJjuXBm_f8DyyXz5NhnV-g6zInzksfSg33yXmqlw== gAAAAABmjiRvdNjOihGYQRjg4dh8QpFas0Qs-xuXwQSP8lwOxetAIenSTUq2m_XZ3oB6l1TjyjCtZWDZ_RR-NN1F18bGeKgvlw== 1997-01-01
